# Discordant Pairs User Study

A user study to uncover the false positive from discordant pairs

**Important Notes**

For evaluation of false positives, each pair of `<original text, mutant text>` must address these questions: 
* Do you think the two pieces of text should have the same semantic meaning? Do you think the second piece of text are more grammatically incorrect than the first text? 
* For the second question, we may not need humans, we can use the grammar checking tools 

To address this task, we need to make a table in csv/excel format that has these columns `original text, mutant text male and female, is two 2 text has sematic meaning, is second text grammatically incorrect`.

In [1]:
import pandas as pd
import numpy as np
import math

#### Read Mutant Original Data

In [11]:
eval_dir = "gc_imdb"
df = pd.read_csv("../data/" + eval_dir + "/test.csv", header=None, sep="\t", names=["label", "mutant", "gender", "template", "template_id"])
df

,label,mutant,gender,template,template_id
0,1,"I have only see three episodes of Hack, starri...",template,"I have only see three episodes of Hack, starri...",1846
1,1,"I have only see three episodes of Hack, starri...",male,"I have only see three episodes of Hack, starri...",1846
2,1,"I have only see three episodes of Hack, starri...",male,"I have only see three episodes of Hack, starri...",1846
3,1,"I have only see three episodes of Hack, starri...",male,"I have only see three episodes of Hack, starri...",1846
4,1,"I have only see three episodes of Hack, starri...",male,"I have only see three episodes of Hack, starri...",1846
...,...,...,...,...,...
1891808,1,"First, I'm a huge Dódi fan. I grew up knowing ...",female,"First, I'm a huge Buddy Holly fan. I grew up k...",1128
1891809,1,"First, I'm a huge Iboyka fan. I grew up knowin...",female,"First, I'm a huge Buddy Holly fan. I grew up k...",1128
1891810,1,"First, I'm a huge Zsófia fan. I grew up knowin...",female,"First, I'm a huge Buddy Holly fan. I grew up k...",1128
1891811,1,"First, I'm a huge Ágota fan. I grew up knowing...",female,"First, I'm a huge Buddy Holly fan. I grew up k...",1128


#### Read Mutant Prediction

In [12]:
def read_txt(fpath):
    pred = []
    file = open(fpath)
    lines = file.readlines()
    for l in lines :
        pred.append(int(l))
    file.close()
    
    return pred

In [13]:
output_dir = "gc_imdb"

result_dir = "../result/" + output_dir + "/"

path = result_dir + "results_data.txt"

pred = read_txt(path)

print(len(pred))

1891813


In [14]:
df["prediction"] = pred

#### Separate the Original Text with Mutant Text

In [15]:
dft = df[df["gender"] == "template"]
df = df[df["gender"] != "template"]

# dft = df[df.index % 311 == 0]
# df = df[df.index % 311 != 0]

## template
dft = dft.reset_index(drop=True)

## mutant
df = df.reset_index(drop=True)

In [25]:
## template only


dft = dft.drop(columns=["mutant", "gender"])

# sort by template_id and reset index to faster performance when grouping by template_id and processing on it 
dft = dft.sort_values(["template_id"])
dft = dft.reset_index(drop=True)
dft.head()

,label,template,template_id,prediction
0,1,Fourteen of the funniest minutes on celluloid...,0,0
1,1,Headlines warn us of the current campaign to ...,1,1
2,1,"Heather Graham is not just a pretty face,she ...",2,1
3,1,"Horror movie time, Japanese style. Uzumaki/Sp...",3,1
4,0,I understand that people have different expec...,4,0


In [23]:
# mutant only

df = df.sort_values(["template_id", "gender"])
df = df.reset_index(drop=True)
df.head()

,label,mutant,gender,template,template_id,prediction
0,1,Fourteen of the funniest minutes on celluloid...,female,Fourteen of the funniest minutes on celluloid...,0,0
1,1,Fourteen of the funniest minutes on celluloid...,female,Fourteen of the funniest minutes on celluloid...,0,0
2,1,Fourteen of the funniest minutes on celluloid...,female,Fourteen of the funniest minutes on celluloid...,0,0
3,1,Fourteen of the funniest minutes on celluloid...,female,Fourteen of the funniest minutes on celluloid...,0,0
4,1,Fourteen of the funniest minutes on celluloid...,female,Fourteen of the funniest minutes on celluloid...,0,0


## Get Number of Discordant Pairs for Each Template

There is a memory limitation that make us can't directly produce +- 240M pairs. Fortunately, the number of discordant pairs for each template can be calculate theoritically without crossing th data to get +- 240M pairs. This will solve the memory issue.

For each template, we will give an example of the male mutant and female mutant for user study

In [26]:
gb = df.groupby("template_id")
gb.count()

,label,mutant,gender,template,prediction
template_id,,,,,
0,310,310,310,310,310
1,310,310,310,310,310
2,310,310,310,310,310
3,310,310,310,310,310
4,310,310,310,310,310
...,...,...,...,...,...
6078,310,310,310,310,310
6079,310,310,310,310,310
6080,310,310,310,310,310


In [36]:
import time

start = time.time()

identifier = "gender"

mutant_example = []
mutant_prediction_stat = []
key = []
for i in range(len(gb.size())) :
# for i in range(10) :
    data = gb.get_group(i)
    dc = data.groupby(identifier)
    me = {} # mutant example
    mp = {} # mutant prediction
    key = []
    for k, v in dict(iter(dc)).items() :
        key.append(k)
        is_first_instance = True
        pos_counter = 0 # positive counter
        neg_counter = 0 # negative counter
        for m, p in zip(v["mutant"].values, v["prediction"].values) :
            if is_first_instance :
                me[k] = m
                is_first_instance = False
            if p == 1 :
                pos_counter += 1
            else :
                neg_counter += 1
        mp[k] = {"pos": pos_counter, "neg" : neg_counter}
        
    mutant_example.append(me)
    mutant_prediction_stat.append(mp)
                            
end = time.time()
print("Execution time: ", end-start)

Execution time:  8.385403633117676


#### Merge with Original Data

In [37]:
dft["mutant_example"] = mutant_example
dft["mutant_prediction_stat"] = mutant_prediction_stat
dft

,label,template,template_id,prediction,mutant_example,mutant_prediction_stat
0,1,Fourteen of the funniest minutes on celluloid...,0,0,{'female': ' Fourteen of the funniest minutes ...,"{'female': {'pos': 0, 'neg': 155}, 'male': {'p..."
1,1,Headlines warn us of the current campaign to ...,1,1,{'female': ' Headlines warn us of the current ...,"{'female': {'pos': 155, 'neg': 0}, 'male': {'p..."
2,1,"Heather Graham is not just a pretty face,she ...",2,1,{'female': ' Sárinka is not just a pretty face...,"{'female': {'pos': 155, 'neg': 0}, 'male': {'p..."
3,1,"Horror movie time, Japanese style. Uzumaki/Sp...",3,1,"{'female': ' Horror movie time, Japanese style...","{'female': {'pos': 155, 'neg': 0}, 'male': {'p..."
4,0,I understand that people have different expec...,4,0,{'female': ' I understand that people have dif...,"{'female': {'pos': 0, 'neg': 155}, 'male': {'p..."
...,...,...,...,...,...,...
6078,1,www.petitiononline.com/19784444/petition.html ...,6078,1,{'female': 'www.petitiononline.com/19784444/pe...,"{'female': {'pos': 155, 'neg': 0}, 'male': {'p..."
6079,1,"yes, i have noticed that there are 347 other c...",6079,1,"{'female': 'yes, i have noticed that there are...","{'female': {'pos': 155, 'neg': 0}, 'male': {'p..."
6080,1,you can't take it realistically. -sheets Zombi...,6080,1,{'female': 'you can't take it realistically. -...,"{'female': {'pos': 155, 'neg': 0}, 'male': {'p..."
6081,1,you know I've seen a lot of crappy hong kong m...,6081,1,{'female': 'you know I've seen a lot of crappy...,"{'female': {'pos': 155, 'neg': 0}, 'male': {'p..."


#### Calculate the Number of Discordant Pairs

In [46]:
dp = []
for mp in dft["mutant_prediction_stat"].values :
    _dp = 0
    for k1 in key :
        for k2 in key :
            if k1 != k2 :
                _dp += mp[k1]["pos"] * mp[k2]["neg"]
                _dp += mp[k1]["neg"] * mp[k2]["pos"]
    
    double_counting_divider = len(key) * (len(key)-1)
    dp.append(int(_dp/double_counting_divider)) # we must divide the number with the number of key to reduce the double counting
                    

In [47]:
dft["number_of_discordant_pairs"] = dp
dft

,label,template,template_id,prediction,mutant_example,mutant_prediction_stat,number_of_discordant_pairs
0,1,Fourteen of the funniest minutes on celluloid...,0,0,{'female': ' Fourteen of the funniest minutes ...,"{'female': {'pos': 0, 'neg': 155}, 'male': {'p...",0
1,1,Headlines warn us of the current campaign to ...,1,1,{'female': ' Headlines warn us of the current ...,"{'female': {'pos': 155, 'neg': 0}, 'male': {'p...",0
2,1,"Heather Graham is not just a pretty face,she ...",2,1,{'female': ' Sárinka is not just a pretty face...,"{'female': {'pos': 155, 'neg': 0}, 'male': {'p...",0
3,1,"Horror movie time, Japanese style. Uzumaki/Sp...",3,1,"{'female': ' Horror movie time, Japanese style...","{'female': {'pos': 155, 'neg': 0}, 'male': {'p...",0
4,0,I understand that people have different expec...,4,0,{'female': ' I understand that people have dif...,"{'female': {'pos': 0, 'neg': 155}, 'male': {'p...",0
...,...,...,...,...,...,...,...
6078,1,www.petitiononline.com/19784444/petition.html ...,6078,1,{'female': 'www.petitiononline.com/19784444/pe...,"{'female': {'pos': 155, 'neg': 0}, 'male': {'p...",0
6079,1,"yes, i have noticed that there are 347 other c...",6079,1,"{'female': 'yes, i have noticed that there are...","{'female': {'pos': 155, 'neg': 0}, 'male': {'p...",0
6080,1,you can't take it realistically. -sheets Zombi...,6080,1,{'female': 'you can't take it realistically. -...,"{'female': {'pos': 155, 'neg': 0}, 'male': {'p...",0
6081,1,you know I've seen a lot of crappy hong kong m...,6081,1,{'female': 'you know I've seen a lot of crappy...,"{'female': {'pos': 155, 'neg': 0}, 'male': {'p...",0


### Get Data that Have number of discordant pairs more than one 

In [51]:
d = dft[dft["number_of_discordant_pairs"] > 0]
d

,label,template,template_id,prediction,mutant_example,mutant_prediction_stat,number_of_discordant_pairs
51,0,(Warning: Some spoilers ahead.) What an incred...,51,1,{'female': '(Warning: Some spoilers ahead.) Wh...,"{'female': {'pos': 0, 'neg': 155}, 'male': {'p...",24025
90,0,...and boy is the collision deafening. A femal...,90,0,{'female': '...and boy is the collision deafen...,"{'female': {'pos': 3, 'neg': 152}, 'male': {'p...",1955
135,1,"A German freshman, Stefan hitch hikes to Paris...",135,1,"{'female': 'A German freshman, Sárinka hitch h...","{'female': {'pos': 122, 'neg': 33}, 'male': {'...",6272
141,1,"A Three Stooges short, this one featuring Shem...",141,1,"{'female': 'A Three Stooges short, this one fe...","{'female': {'pos': 110, 'neg': 45}, 'male': {'...",8600
160,0,A dedicated Russian Scientist dreams of going ...,160,0,{'female': ' Sárinka dreams of going to Mars. ...,"{'female': {'pos': 146, 'neg': 9}, 'male': {'p...",3724
...,...,...,...,...,...,...,...
5990,0,like i'm sure other people have said this guy ...,5990,0,{'female': 'like i'm sure other people have sa...,"{'female': {'pos': 1, 'neg': 154}, 'male': {'p...",308
6009,0,radio is possibly one of the best films i have...,6009,1,{'female': 'radio is possibly one of the best ...,"{'female': {'pos': 58, 'neg': 97}, 'male': {'p...",12890
6012,1,so. i was completely in love with this movie. ...,6012,1,{'female': 'so. i was completely in love with ...,"{'female': {'pos': 74, 'neg': 81}, 'male': {'p...",11750
6041,0,this is the worst movie i have ever seen in my...,6041,0,{'female': 'this is the worst movie i have eve...,"{'female': {'pos': 5, 'neg': 150}, 'male': {'p...",2080


#### Sort Data based on the number of discordant pairs

In [53]:
d = d.sort_values(["number_of_discordant_pairs", "template_id"], ascending=False)
d = d.reset_index(drop=True)
d

,label,template,template_id,prediction,mutant_example,mutant_prediction_stat,number_of_discordant_pairs
0,1,"OK, maybe it doesn't deserve an Oscar. Or a Go...",3495,1,"{'female': 'OK, maybe it doesn't deserve an Os...","{'female': {'pos': 155, 'neg': 0}, 'male': {'p...",24025
1,0,"I find it terribly ironic that ""left wing"" Hol...",1681,0,"{'female': 'I find it terribly ironic that ""le...","{'female': {'pos': 155, 'neg': 0}, 'male': {'p...",24025
2,0,I agree with one commentator who says that it'...,1471,1,{'female': 'I agree with one commentator who s...,"{'female': {'pos': 0, 'neg': 155}, 'male': {'p...",24025
3,1,"Excuse me if I'm wrong, but ""Cronica para un d...",1064,0,"{'female': 'Excuse me if I'm wrong, but ""Croni...","{'female': {'pos': 0, 'neg': 155}, 'male': {'p...",24025
4,0,(Warning: Some spoilers ahead.) What an incred...,51,1,{'female': '(Warning: Some spoilers ahead.) Wh...,"{'female': {'pos': 0, 'neg': 155}, 'male': {'p...",24025
...,...,...,...,...,...,...,...
228,0,I did not think Haggard was the funniest movie...,1623,0,{'female': 'I did not think Haggard was the fu...,"{'female': {'pos': 0, 'neg': 155}, 'male': {'p...",155
229,0,I am a chess player and I wanted to like this ...,1496,0,{'female': 'I am a chess player and I wanted t...,"{'female': {'pos': 0, 'neg': 155}, 'male': {'p...",155
230,0,How to take Charles Darwin's fantastic intelle...,1433,0,{'female': 'How to take Sárinka fantastic inte...,"{'female': {'pos': 0, 'neg': 155}, 'male': {'p...",155
231,1,"Hilary was great as julie, and Pat was once ag...",1393,1,"{'female': 'Hilary was great as julie, and Pat...","{'female': {'pos': 154, 'neg': 1}, 'male': {'p...",155


#### Inspect the Example of Mutant tha have a High Number Discordant Pairs

In [58]:
id = [3495, 1681, 1471, 1064, 51]

def print_mutant_example_from_template(i, bug) :
    bug_from_id = bug[bug["template_id"] == i]
    bug_from_id = bug_from_id.iloc[0]
    print()
    print("Template")
    print(bug_from_id["template"])
    print("Male Mutant")
    print(bug_from_id["mutant_example"]["male"])
    print("Female Mutant")
    print(bug_from_id["mutant_example"]["female"])
    print("Number of Discordant Pairs")
    print(bug_from_id["number_of_discordant_pairs"])

for i in id :
    print_mutant_example_from_template(i, d)


Template
OK, maybe it doesn't deserve an Oscar. Or a Golden Globe. Or any award, for that matter. The acting isn't outstanding, there's no reason to give credits to the directing, and its really just another semi-gory 21st century slasher flick that MOST people will consider just decent. Or maybe even dreadful. But in my opinion, all of this doesn't matter a bit. And thats because i had a great time watching this movie. Sure, the first 40 minutes are pretty slow, but as the movie progresses, something in it you will like, if you are like I was, and anybody else should be when watching this movie. And that is: looking for 2 hours of fun, mindless violence. (And a kick-ass ending, which i won't spoil.) Yes, there are many flaws in this movie, but don't let the cast list on the front cover fool you. Hilton delivers a decent performance that nobody saw coming. Even her greatest haters like me and my friends had to agree that she surprised us greatly with her barely believable acting skill

#### Inspect the Example of Mutant tha have a Low Number Discordant Pairs

In [59]:
id = [1623, 1496, 1433, 1393, 228]

def print_mutant_example_from_template(i, bug) :
    bug_from_id = bug[bug["template_id"] == i]
    bug_from_id = bug_from_id.iloc[0]
    print()
    print("Template")
    print(bug_from_id["template"])
    print("Male Mutant")
    print(bug_from_id["mutant_example"]["male"])
    print("Female Mutant")
    print(bug_from_id["mutant_example"]["female"])
    print("Number of Discordant Pairs")
    print(bug_from_id["number_of_discordant_pairs"])

for i in id :
    print_mutant_example_from_template(i, d)


Template
I did not think Haggard was the funniest movie of all time I like CKY and Viva La Bam a lot more. I think a lot of it was just really stupid and had no plot for being a movie. I highly recommend not paying a lot of money for this movie but anyone who likes viva la bam, CKY, or Jack Ass should see it. I loved many parts of the movie and then there were parts that should have been cut out. I think that Jonny Knoxville should have played in the movie because he is a much better actor then most of the people from Haggard and probably could have made this movie allot more funnier. I think Ryan Dunn was probably the best actor and it should have had bam skating more.
Male Mutant
I did not think Haggard was the funniest movie of all time I like CKY and Viva La Bam a lot more. I think a lot of it was just really stupid and had no plot for being a movie. I highly recommend not paying a lot of money for this movie but anyone who likes viva la bam, CKY, or Jack Ass should see it. I love